In [1]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd

load_dotenv()
apikey = os.environ.get("apikey")

snp500 = pd.DataFrame(fmpsdk.sp500_constituent(apikey))

In [2]:
from strategy import *

initial_holdings = [Holding(symbol = symbol) for symbol in snp500['symbol']]
start_date = '2017-03-01'
end_date = '2019-03-01'
strategist = Strategist(database_list=[], initial_holdings=initial_holdings, start_date=start_date, end_date=end_date)

filter_1  = Filter(selector_sequence_union=[]) 
strategist.apply_filter(filter = filter_1, date = '2018-03-01')

In [3]:
strategist.state

[Segment(start_date='2018-03-01', end_date='2019-03-01', holdings=[]),
 Segment(start_date='2017-03-01', end_date='2018-03-01', holdings=[Holding(symbol='MMM'), Holding(symbol='AOS'), Holding(symbol='ABT'), Holding(symbol='ABBV'), Holding(symbol='ABMD'), Holding(symbol='ACN'), Holding(symbol='ATVI'), Holding(symbol='ADM'), Holding(symbol='ADBE'), Holding(symbol='ADP'), Holding(symbol='AAP'), Holding(symbol='AES'), Holding(symbol='AFL'), Holding(symbol='A'), Holding(symbol='APD'), Holding(symbol='AKAM'), Holding(symbol='ALK'), Holding(symbol='ALB'), Holding(symbol='ARE'), Holding(symbol='ALGN'), Holding(symbol='ALLE'), Holding(symbol='LNT'), Holding(symbol='ALL'), Holding(symbol='GOOGL'), Holding(symbol='GOOG'), Holding(symbol='MO'), Holding(symbol='AMZN'), Holding(symbol='AMCR'), Holding(symbol='AMD'), Holding(symbol='AEE'), Holding(symbol='AAL'), Holding(symbol='AEP'), Holding(symbol='AXP'), Holding(symbol='AIG'), Holding(symbol='AMT'), Holding(symbol='AWK'), Holding(symbol='AMP'), Ho

In [7]:
import pickle
with open('static/data.pickle', 'rb') as f:
    dict_symbol_result = pickle.load(f)

In [13]:
import pandas as pd
df_financial_ratios =  pd.DataFrame(dict_symbol_result['EXR'])

In [22]:
a_list = ['a', 'b', 'c']
gen = [x for x in a_list if x  == 'a']
gen

<generator object <genexpr> at 0x000001F0671217E0>